In [ ]:
!pip install pysrt requests sentencepiece transformers==4.33 sacremoses  sacrebleu

In [ ]:
import requests
import pysrt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import re
from google.colab import files



In [ ]:
model_name =  "zaanind/nllb-ensi-v1-tuning" #"zaanind/nllb-ensi-v1-tuning-subs" #"facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
def dotranslate(text):
  inputs = tokenizer(text, return_tensors="pt")
  translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["sin_Sinh"])
  out = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
  return out


In [ ]:

print("<< Upload Subtitle file to translate >>")
uploaded = files.upload()

for fn in uploaded.keys():
  filenm = fn
  print(fn)


print("<<Translating subtitle in progress>>")

# Read the subtitles from a file using pysrt
subtitles = pysrt.open(filenm)

# Iterate through each subtitle line and translate
for subtitle in subtitles:
    input_text = subtitle.text

    translated_text = dotranslate(input_text)
    print(translated_text)

    # Update the subtitle with the translated text
    subtitle.text = translated_text






In [ ]:
# @title Download Translated SRT

fname = f"translated_{filenm}"
# Save the modified subtitles to a new file
subtitles.save(fname)

print(f"<<downloading your file: {fname} \n if not, download from file manager>>")

files.download(fname)

In [ ]:
# @title batch translator

batch_size = 3 #num of lines per time

print("<< Upload Subtitle file to translate >>")
uploaded = files.upload()

for fn in uploaded.keys():
  filenm = fn
 # print(fn)


print("<<Translating subtitle in progress>>")


# Read the subtitles from a file using pysrt
subtitles = pysrt.open(filenm)


for subtitle in subtitles:
    input_text = subtitle.text
    text = f"<sb>{input_text}</sb>"
    subtitle.text = text




for i in range(0, len(subtitles), batch_size):
  batch = subtitles[i:i + batch_size]
  sendai = ""
  for item in batch:
    sendai = sendai + item.text
  outed = dotranslate(sendai)
  print(outed)
  pattern = r"<sb>(.*?)</sb>"
  spes = re.findall(pattern, outed)
  print(len(spes))
  for item,trted in zip(batch,spes):
    item.text = trted



fname = f"translated_{filenm}"
# Save the modified subtitles to a new file
subtitles.save(fname)

print(f"<<downloading your file: {fname} \n if not, download from file manager>>")

files.download(fname)


In [ ]:
def translate(text): #define function for generation
  inputs = tokenizer(text, return_tensors="pt")
  translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["sin_Sinh"])
  out = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
  return out


translate("hello how are you?") #translate